# Laboratorio: Modelos del lenguaje con RNNs

En este laboratorio, vamos a entrenar un modelo del lenguaje basado en caracteres con Recurrent Neural Networks. Asimismo, utilizaremos el modelo para generar texto. En particular, alimentaremos nuestro modelo con obras de la literatura clásica en castellano para obtener una red neuronal que sea capaz de "escribir" fragmentos literarios.

Los entrenamientos en esta laboratorio para obtener un modelo de calidad podrían tomar cierto tiempo (5-10 minutos por epoch), por lo que se aconseja empezar a trabajar pronto. El uso de GPUs no ayuda tanto con LSTMs como con CNNs, por lo que si tenéis máquinas potentes en casa es posible que podáis entrenar más rápido o a la misma velocidad que en Colab. En todo caso, la potencia de Colab es más que suficiente para completar este laboratorio con éxito.

<center><img src="https://upload.wikimedia.org/wikipedia/commons/d/d8/El_ingenioso_hidalgo_don_Quijote_de_la_Mancha.jpg" style="text-align: center" height="300px"></center>

El dataset a utilizar consistirá en un archivo de texto con el contenido íntegro en castellano antiguo de El Ingenioso Hidalgo Don Quijote de la Mancha, disponible de manera libre en la página de [Project Gutenberg](https://www.gutenberg.org). Asimismo, como apartado optativo en este laboratorio se pueden utilizar otras fuentes de texto. Aquí podéis descargar los datos a utilizar de El Quijote y un par de obras adicionales:

[El ingenioso hidalgo Don Quijote de la Mancha (Miguel de Cervantes)](https://onedrive.live.com/download?cid=C506CF0A4F373B0F&resid=C506CF0A4F373B0F%219424&authkey=AH0gb-qSo5Xd7Io)

[Compilación de obras teatrales (Calderón de la Barca)](https://onedrive.live.com/download?cid=C506CF0A4F373B0F&resid=C506CF0A4F373B0F%219433&authkey=AKvGD6DC3IRBqmc)

[Trafalgar (Benito Pérez Galdós)](https://onedrive.live.com/download?cid=C506CF0A4F373B0F&resid=C506CF0A4F373B0F%219434&authkey=AErPCAtMKOI5tYQ)

Como ya deberíamos de estar acostumbrados en problemas de Machine Learning, es importante echar un vistazo a los datos antes de empezar.

## 1. Carga y procesado del texto

Primero, vamos a descargar el libro e inspeccionar los datos. El fichero a descargar es una versión en .txt del libro de Don Quijote, a la cual se le han borrado introducciones, licencias y otras secciones para dejarlo con el contenido real de la novela.

In [18]:
import numpy as np 
import keras
import matplotlib.pyplot as plt
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import random
import io

path = keras.utils.get_file(
    fname="don_quijote.txt", 
    origin="https://onedrive.live.com/download?cid=C506CF0A4F373B0F&resid=C506CF0A4F373B0F%219424&authkey=AH0gb-qSo5Xd7Io"
)

Una vez descargado, vamos a leer el contenido del fichero en una variable. Adicionalmente, convertiremos el contenido del texto a minúsculas para ponérselo un poco más fácil a nuestro modelo (de modo que todas las letras sean minúsculas y el modelo no necesite diferenciar entre minúsculas y mayúsculas).

**1.1.** Leer todo el contenido del fichero en una única variable ***text*** y convertir el string a minúsculas

In [19]:
text = ''

# Abre el archivo de texto en 'path' en modo de lectura ('r') con la codificación 'utf8'
# y asignar el objeto de archivo devuelto a la variable 'f'
with open(path, 'r', encoding="utf8") as f:
    
    # Lee todo el contenido del archivo de texto con el método 'read()',
    # convierte el texto a minúsculas utilizando el método 'lower()'
    # y asigna el resultado a la variable 'text'
    text = f.read().lower()

Limpiamos los caracteres que no son necesarios del texto:

In [20]:
# Cadena de caracteres inválidos que se eliminarán del texto
invalid_chars = "_-+.,:;'\"«»¡!¿?()[]{}"

# Tabla de traducción que convierte caracteres inválidos a caracteres vacíos
change_chars = str.maketrans("", "", invalid_chars)

# Tabla de traducción para eliminar los caracteres inválidos de text
text = text.translate(change_chars)

Podemos comprobar ahora que efectivamente nuestra variable contiene el resultado deseado, con el comienzo tan característico del Quijote.

In [21]:
# Longitud del texto
print("Longitud del texto: {}".format(len(text)))

# Imprime los primeros 300 caracteres
print(text[0:300])

Longitud del texto: 2005133
capítulo primero que trata de la condición y ejercicio del famoso hidalgo
don quijote de la mancha


en un lugar de la mancha de cuyo nombre no quiero acordarme no ha mucho
tiempo que vivía un hidalgo de los de lanza en astillero adarga antigua
rocín flaco y galgo corredor una olla de algo más vaca 


## 2. Procesado de los datos

Una de las grandes ventajas de trabajar con modelos que utilizan caracteres en vez de palabras es que no necesitamos tokenizar el texto (partirlo palabra a palabra). Nuestro modelo funcionará directamente con los caracteres en el texto, incluyendo espacios, saltos de línea, etc.

Antes de hacer nada, necesitamos procesar el texto en entradas y salidas compatibles con nuestro modelo. Como sabemos, un modelo del lenguaje con RNNs acepta una serie de caracteres y predice el siguiente carácter en la secuencia.

* "*El ingenioso don Qui*" -> predicción: **j**
* "*El ingenioso don Quij*" -> predicción: **o**

De modo que la entrada y la salida de nuestro modelo necesita ser algo parecido a este esquema. En este punto, podríamos usar dos formas de preparar los datos para nuestro modelo.

1. **Secuencia a secuencia**. La entrada de nuestro modelo sería una secuencia y la salida sería esa secuencia trasladada un caracter a la derecha, de modo que en cada instante de tiempo la RNN tiene que predecir el carácter siguiente. Por ejemplo:

>* *Input*:   El ingenioso don Quijot 
>* *Output*: l ingenioso don Quijote

2. **Secuencia a carácter**. En este variante, pasaríamos una secuencia de caracteres por nuestra RNN y, al llegar al final de la secuencia, predeciríamos el siguiente carácter.

>* *Input*:   El ingenioso don Quijot 
>* *Output*: e

En este laboratorio, por simplicidad, vamos a utilizar la segunda variante.

De este modo, a partir del texto, hemos de generar nuestro propio training data que consista en secuencias de caracteres con el siguiente carácter a predecir. Para estandarizar las cosas, utilizaremos secuencias de tamaño *SEQ_LENGTH* caracteres (un hiperparámetro que podemos elegir nosotros).



#### 2.1. Obtención de los caracteres y mapas de caracteres

Antes que nada, necesitamos saber qué caracteres aparecen en el texto, ya que tendremos que diferenciarlos mediante un índice de 0 a *num_chars* - 1 en el modelo. Obtener:
 

1.   Número de caracteres únicos que aparecen en el texto.
2.   Diccionario que asocia char a índice único entre 0 y *num_chars* - 1. Por ejemplo, {'a': 0, 'b': 1, ...}
3.   Diccionario reverso de índices a caracteres: {0: 'a', 1: 'b', ...}


In [22]:
# Lista de caracteres individuales de text
all_chars = list(text)

# Longitud de la lista resultante
len(all_chars)

2005133

##### **Eliminación de caracteres especiales y creación de diccionario con los caracteres diferentes**

In [23]:
# Lista con los caracteres únicos del texto
chars = sorted(list(set(text)))

# Diccionario que asocia cada caracter a índice único
char_index = {c: i for i, c in enumerate(chars) if not c in invalid_chars}

# Diccionario reverso de índices a caracteres
index_char = {c: i for i, c in char_index.items()}

In [24]:
char_index

{'\n': 0,
 ' ': 1,
 '0': 2,
 '1': 3,
 '2': 4,
 '3': 5,
 '4': 6,
 '5': 7,
 '6': 8,
 '7': 9,
 'a': 10,
 'b': 11,
 'c': 12,
 'd': 13,
 'e': 14,
 'f': 15,
 'g': 16,
 'h': 17,
 'i': 18,
 'j': 19,
 'l': 20,
 'm': 21,
 'n': 22,
 'o': 23,
 'p': 24,
 'q': 25,
 'r': 26,
 's': 27,
 't': 28,
 'u': 29,
 'v': 30,
 'w': 31,
 'x': 32,
 'y': 33,
 'z': 34,
 'à': 35,
 'á': 36,
 'é': 37,
 'í': 38,
 'ï': 39,
 'ñ': 40,
 'ó': 41,
 'ù': 42,
 'ú': 43,
 'ü': 44}

In [25]:
len(chars)

45

#### 2.2. Obtención de secuencias de entrada y carácter a predecir

Ahora, vamos a obtener las secuencias de entrada en formato texto y los correspondientes caracteres a predecir. Para ello, recorrer el texto completo leído anteriormente, obteniendo una secuencia de SEQ_LENGTH caracteres y el siguiente caracter a predecir. Una vez hecho, desplazarse un carácter a la izquierda y hacer lo mismo para obtener una nueva secuencia y predicción. Guardar las secuencias en una variable ***sequences*** y los caracteres a predecir en una variable ***next_chars***.

Por ejemplo, si el texto fuera "Don Quijote" y SEQ_LENGTH fuese 5, tendríamos

* *sequences* = ["Don Q", "on Qu", "n Qui", " Quij", "Quijo", "uijot"]
* *next_chars* = ['u', 'i', 'j', 'o', 't', 'e']

In [26]:
# Definición del tamaño de las secuencias
SEQ_LENGTH = 10

sequences = []
next_chars = []

step = 1 # Siguiente sentencia desplazada una palabra la izquierda
for i in range(0, len(all_chars)-SEQ_LENGTH, step):
    # Añade la secuencia actual a la lista de secuencias
    sequences.append(all_chars[i:i+SEQ_LENGTH])
    # Añade el siguiente caracter después de la secuencia actual a la lista de caracteres siguientes
    next_chars.append(all_chars[i+SEQ_LENGTH])

In [27]:
sequences[-1]

['a', 'l', 'e', '\n', '\n', '\n', '\n', 'f', 'i', 'n']

In [28]:
next_chars[-1]

'\n'

Indicar el tamaño del training set que acabamos de generar.

In [29]:
print("Tamaño del conjunto de entrenamiento:", len(sequences))

Tamaño del conjunto de entrenamiento: 2005123


Como el Quijote es muy largo y tenemos muchas secuencias, podríamos encontrar problemas de memoria. Por ello, vamos a elegir un número máximo de ellas. Si estás corriendo esto localmente y tienes problemas de memoria, puedes reducir el tamaño aún más, pero ten cuidado porque, a menos datos, peor calidad del modelo.

In [30]:
# Definición del máximo número de secuencias a usar
MAX_SEQUENCES = len(sequences)

# Conversión de las listas a numpy arrays para su procesamiento
sequences, next_chars = np.array(sequences), np.array(next_chars)
# Selecció del número máximo de secuencias para usar
sequences, next_chars = list(sequences[:MAX_SEQUENCES]), list(next_chars[:MAX_SEQUENCES])

In [31]:
sequences[-1]

array(['a', 'l', 'e', '\n', '\n', '\n', '\n', 'f', 'i', 'n'], dtype='<U1')

In [32]:
next_chars[-1]

'\n'

#### 2.3. Obtención de input X y output y para el modelo

Finalmente, a partir de los datos de entrenamiento que hemos generado vamos a crear los arrays de datos X e y que pasaremos a nuestro modelo.

Para ello, **NO** vamos a utilizar *one-hot encoding* para nuestros caracteres. Por ejemplo, si sólo tuviéramos 4 caracteres (a, b, c, d), las representaciones serían: (1, 0, 0, 0), (0, 1, 0, 0), (0, 0, 1, 0) y (0, 0, 0, 1).

De este modo, **X** tendrá shape *(num_sequences, seq_length, num_chars)* e **y** tendrá shape *(num_sequences, num_chars)*.

Sino que usaremos **Embedings**

**One-hot Encoding**: consiste en crear una columna binaria (que solo puede contener los valores 0 o 1) para cada valor único que exista en la variable categórica que estamos codificando, y marcar con un 1 la columna correspondiente al valor presente en cada registro, dejando las demás columnas con un valor de 0. Una desventaja de este método es que estamos aumentando la dimensionalidad del conjunto de datos (es decir, aumentando el número de columnas o características categóricas a partir de las cuales entrenar el modelo), lo que puede resultar problemático si el número de muestras de las que se dispone no es suficientemente elevado.

**Definición de X e Y**:

In [33]:
# Definición de X e Y

# Obtener el número de secuencias en la lista 'sequences'
NUM_SEQUENCES = len(sequences)

# Matriz 'X' llena de ceros con las dimensiones adecuadas
# 'NUM_SEQUENCES' representa el número de secuencias en 'sequences'
# 'SEQ_LENGTH' representa la longitud máxima de cada secuencia
# 'len(chars)' representa el número de caracteres únicos en el conjunto de datos
X = np.zeros((NUM_SEQUENCES, SEQ_LENGTH, len(chars)))

# Matriz 'y' llena de ceros con las dimensiones adecuadas
# 'NUM_SEQUENCES' representa el número de secuencias en 'sequences'
# 'len(chars)' representa el número de caracteres únicos en el conjunto de datos
y = np.zeros((NUM_SEQUENCES, len(chars)))

In [34]:
for k in range(NUM_SEQUENCES):
    # Obtiene el siguiente caracter de cada secuencia
    c = next_chars[k]
    # Establece en 1 el índice correspondiente a este caracter 
    # en el vector objetivo 'y'  para la secuencia k. 
    y[k, char_index[c]] = 1
    
# 'char_index': diccionario que asocia cada caracter a un índice único.

In [35]:
for k in range(NUM_SEQUENCES):
    for w in range(SEQ_LENGTH):
        # Obtiene el carácter en la posición 'w' de la secuencia 'k'
        c = sequences[k][w]
        # Establece en 1 el índice correspondiente a este carácter 
        # en la matriz 'X' para la secuencia k y posición w.
        X[k, w, char_index[c]] = 1

In [36]:
print(X)

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]


In [37]:
print(y)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


## 3. Definición del modelo y entrenamiento

Una vez tenemos ya todo preparado, es hora de definir el modelo. Define un modelo que utilice una **LSTM** con **128 unidades internas**. Si bien el modelo puede definirse de una manera más compleja, para empezar debería bastar con una LSTM más una capa Dense con el *softmax* que predice el siguiente caracter a producir. Adam puede ser una buena elección de optimizador.

Una vez el modelo esté definido, entrénalo un poco para asegurarte de que la loss es decreciente. No es necesario guardar la salida de este entrenamiento en el entregable final, ya que vamos a hacer el entrenamiento más informativo en el siguiente punto.

In [38]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

model = Sequential()
# Capa LSTM con 128 unidades
model.add(LSTM(128, input_shape=(SEQ_LENGTH, len(char_index))))
# Capa densa con 1024 unidades y función de activación relu
model.add(Dense(1024, activation='relu'))
# Capa densa con tantas unidades como caracteres únicos y función de activación softmax
model.add(Dense(len(chars), activation='softmax'))
# Muestra un resumen del modelo
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               89088     
                                                                 
 dense (Dense)               (None, 1024)              132096    
                                                                 
 dense_1 (Dense)             (None, 45)                46125     
                                                                 
Total params: 267,309
Trainable params: 267,309
Non-trainable params: 0
_________________________________________________________________
None


In [39]:
# Compilación del modelo
model.compile(loss='MSE', optimizer='adam', metrics=['accuracy'])
# Entrenamiento del modelo y registro del historial
history = model.fit(X, y, validation_split=0.05, batch_size=128, epochs=5, shuffle=True).history

Epoch 1/5
14882/14882 [==============================] - 305s 20ms/step - loss: 0.0140 - accuracy: 0.4976 - val_loss: 0.0128 - val_accuracy: 0.5475
Epoch 2/5
14882/14882 [==============================] - 294s 20ms/step - loss: 0.0122 - accuracy: 0.5700 - val_loss: 0.0121 - val_accuracy: 0.5756
Epoch 3/5
14882/14882 [==============================] - 295s 20ms/step - loss: 0.0117 - accuracy: 0.5869 - val_loss: 0.0118 - val_accuracy: 0.5852
Epoch 4/5
14882/14882 [==============================] - 294s 20ms/step - loss: 0.0115 - accuracy: 0.5967 - val_loss: 0.0116 - val_accuracy: 0.5926
Epoch 5/5
14882/14882 [==============================] - 292s 20ms/step - loss: 0.0113 - accuracy: 0.6026 - val_loss: 0.0115 - val_accuracy: 0.5967


**X a predecir**:

In [79]:
# X a predecir
X_pred = np.zeros((1, SEQ_LENGTH, len(chars)))
X_pred[0,0] = X[0,0]
X_pred[0,1] = X[0,1]
X_pred[0,2] = X[0,2]
X_pred[0,3] = X[0,3]
X_pred[0,4] = X[0,4]
X_pred[0,5] = X[0,5]
X_pred[0,6] = X[0,6]
X_pred[0,7] = X[0,7]
X_pred[0,8] = X[0,8]
X_pred[0,9] = X[0,9]

# Predicción del siguiente caracter
prediccion = model.predict(X_pred, batch_size=32, verbose=0)

In [80]:
prediccion

array([[2.79704749e-04, 7.33007537e-03, 2.48895347e-04, 2.28706980e-04,
        9.88617321e-05, 2.62180081e-04, 1.29823835e-04, 1.16870149e-04,
        1.66574100e-04, 1.60652795e-04, 1.61076561e-01, 2.07205987e-04,
        8.22628965e-04, 2.24684575e-03, 2.21271232e-01, 2.81279237e-04,
        1.25042989e-03, 5.54673315e-04, 2.95986608e-02, 1.29991525e-03,
        2.68333927e-02, 1.06988288e-03, 2.76961620e-03, 2.95817167e-01,
        4.17921838e-04, 2.34358246e-04, 1.18135326e-01, 4.75485390e-03,
        1.40146585e-03, 9.67003256e-02, 6.14372082e-04, 8.20773639e-05,
        4.05388360e-04, 9.49737674e-04, 8.10292040e-05, 1.62865865e-04,
        2.44862447e-03, 3.98424407e-03, 3.35194683e-03, 2.38597728e-04,
        2.63080263e-04, 8.74996930e-03, 3.02797445e-04, 2.50580744e-03,
        9.34070849e-05]], dtype=float32)

In [81]:
# Caracter asociado al índice de mayor valor, en el vector de predicción, en el diccionario
index_char[np.argmax(prediccion)]

'o'

In [88]:
index_char[np.argmax(X[1,0])]

'a'

In [89]:
# Evalua el rendimiento del modelo en el conjunto de datos de entrenamiento (X,y)
results = model.evaluate(X, y)

62661/62661 [==============================] - 298s 5ms/step - loss: 0.0111 - accuracy: 0.6096


##### Predicción para una frase

Para ver cómo evoluciona nuestro modelo del lenguaje, vamos a generar texto según va entrenando. Para ello, vamos a programar una función que, utilizando el modelo en su estado actual, genere texto, con la idea de ver cómo se va generando texto al entrenar cada epoch.

In [114]:
# Definición de la función generate_text
def generate_text(length):
    X_pred[0,0] = X[0,0]
    X_pred[0,1] = X[0,1]
    X_pred[0,2] = X[0,2]
    X_pred[0,3] = X[0,3]
    X_pred[0,4] = X[0,4]
    X_pred[0,5] = X[0,5]
    X_pred[0,6] = X[0,6]
    X_pred[0,7] = X[0,7]
    X_pred[0,8] = X[0,8]
    X_pred[0,9] = X[0,9]
    # Itera por cada carácter en el texto generado
    for t in range(length):
        # Predicción con el modelo entrenado y se obtiene el siguiente carácter
        prediccion = model.predict(X_pred, batch_size=32, verbose=0)
        next_pred = index_char[np.argmax(prediccion)]
        # Se desplazan los caracteres en X_pred para poder añadir el siguiente carácter generado
        for k in range(10):
             X_pred[0, k-1] = X_pred[0, k]
        # Imprime el siguiente carácter generado
        print(next_pred)

In [115]:
# Generación de las siguiente 10 letras
generate_text(10)

o
o
b
i
s
u
l
o
 
 


## Entregable

Completa los apartados anteriores para entrenar modelos del lenguaje que sean capaces de generar texto con cierto sentido. Comentar los resultados obtenidos y cómo el modelo va mejorando época a época. Comentar las diferencias apreciadas al utilizar diferentes valores de temperatura. Entregar al menos la salida de un entrenamiento completo con los textos generados época a época.

El objetivo no es conseguir generar pasajes literarios con coherencia, sino obtener lenguaje que se asemeje en cierta manera a lo visto en el texto original y donde las palabras sean reconocibles como construcciones en castellano. Como ejemplo de lo que se puede conseguir, este es el resultado de generar texto después de 10 epochs y con temperature 0.2:


```
-----> Epoch: 10 - Generando texto con temperature 0.2
Seed: o le cautivaron y rindieron el
Texto generado: o le cautivaron y rindieron el caballero de la caballería de la mano de la caballería del cual se le dijo:

-¿quién es el verdad de la caballería de la caballería de la caballería de la caballería de la caballería, y me ha de habían de la mano que el caballero de la mano de la caballería. y que no se le habían de la mano de la c

```

Asimismo, se proponen los siguientes aspectos opcionales para conseguir nota extra:

*   Experimentar con los textos de teatro en verso de Calderón de la Barca (¿es capaz el modelo de aprender las estructuras del teatro en verso?) o con alguno de los otros textos disponibles. También se puede probar con textos de vuestra elección.
*   Experimentar con distintos valores de SEQ_LENGTH.
*   Experimentar con los hiperparámetros del modelo o probar otro tipo de modelos como GRUs o *stacked* RNNs (RNNs apiladas).
*   Experimentar utilizando embeddings en vez de representaciones one-hot.
*   (Difícil) Entrenar un modelo secuencia a secuencia en vez de secuencia a carácter.


